<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Planning-new-network" data-toc-modified-id="Planning-new-network-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Planning new network</a></span></li></ul></div>

# Planning new network
- 2 limiting factors from last model:
    - data
        - 2,000 images is good, but i don't think augmentation will lead to great learning
        - use something like AT&Tfaces, which has ~40 'users' but 8 photos of each user
            - this is few-shot learning after all, the simulated extra shots may not be giving me the desired output
            - then again, it could also be:
    - architecture size
        - 6 layers, mixture of dense+conv+flattening. messed around with different activations
        - i think i should try changing this first, because it's something i want to learn for future models as well
- Model updates:
    - transfer learning
        - try importing VGGnet, Facenet etc and running
        - learn Sagemaker to run it so i'm not limited by local computing power
